In [2]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn


In [3]:
from data.data_generator import get_data, preprocess
from src.my_dnn import create_dnn
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow import keras
import keras_tuner as kt
import tensorflow as tf

In [6]:
print(tf.__version__)
#print(tf.keras.__version__)
print(kt.__version__)
print(np.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

2.15.0
1.4.7
1.26.4
GPU is NOT AVAILABLE


In [ ]:
%tensorboard --logdir logs

In [7]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=80, training=True)
val_data = preprocess(x_val, y_val, batch_size=20, training=False)

In [1]:
403.629/60

6.72715

In [8]:
model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=100,
        num_layers=10,
        num_neurons=20,
        beta=80,
        gamma=80,
        delta=1,
    )

optimizer = keras.optimizers.SGD(learning_rate=np.exp(-14))

model.compile(
    optimizer=optimizer, loss="mean_squared_error"
)

model.fit(train_data, epochs=10, validation_data=val_data)


Epoch 1/10



OperatorNotAllowedInGraphError: in user code:

    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\my_dnn.py", line 148, in train_step
        if tf.linalg.global_norm(vars_diff) > self.delta:

    OperatorNotAllowedInGraphError: Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this

In [12]:
def model_builder(hp):
    hp_units = hp.Int("units", min_value=10, max_value=40, step=10)
    hp_layers = hp.Int("layers", min_value=5, max_value=20, step=5)
    hp_nets = hp.Int("networks", min_value=80, max_value=6 * 80, step=80)
    hp_beta = hp.Float('beta', min_value=20, max_value=80)
    hp_gamma = hp.Float('gamma', min_value=30, max_value=80)
    hp_learning_rate = hp.Float(
        "learning_rate", min_value=np.exp(-16), max_value=np.exp(-14), sampling="log"
    )

    # Hier nutzen Sie Ihre angepasste `create_dnn` Funktion mit den hp-Argumenten
    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=hp_nets,
        num_layers=hp_layers,
        num_neurons=hp_units,
        beta=hp_beta,
        gamma=hp_gamma,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )

    return model

In [ ]:
tuner = kt.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='rs_tuning'
)

# Starten des Tuning-Prozesses
tuner.search(x_train, y_train, epochs=250, validation_data=(x_val, y_val))

# Abrufen der besten Hyperparameter
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [9]:
print(f"""
Die besten Hyperparameter sind:
- Anzahl der Netzwerke: {best_hps.get('networks')}
- Anzahl der Schichten: {best_hps.get('layers')}
- Anzahl der Neuronen: {best_hps.get('units')}
- Delta: {best_hps.get('learning_rate')}
""")


Die besten Hyperparameter sind:
- Anzahl der Netzwerke: 160
- Anzahl der Schichten: 10
- Anzahl der Neuronen: 20
- Delta: 4.156935364128902e-07



In [4]:
def train_and_evaluate_dnn(model, train_data, test_data, epochs=75):
    """
    Trains the model on the given data and evaluates its performance.
    """
    model.fit(train_data, epochs=epochs, verbose=0)
    mse, mae = model.evaluate(test_data, verbose=0)
    return mse, mae

In [5]:
mses = []  # Initialize empty list to store MSEs
maes = []  # Initialize empty list to store MAEs
for _ in range(1):
    x_train, y_train = get_data(
        regression_func, x_dim=input_dim, num_samples=100, sigma=0.05
    )
    x_test, y_test = get_data(
        regression_func, x_dim=input_dim, num_samples=10**5, sigma=0.05
    )

    # Preprocess data
    train_data = preprocess(x_train, y_train, batch_size=100, training=True)
    test_data = preprocess(x_test, y_test, batch_size=100, training=False)

    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=160,
        num_layers=10,
        num_neurons=20,
        beta=80,
        gamma=80,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=np.exp(-15))

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )
    mse, mae = train_and_evaluate_dnn(model, train_data, test_data, epochs=500)
    mses.append(mse)
    maes.append(mae)

NotImplementedError: in user code:

    File "c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\my_dnn.py", line 176, in train_step  *
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 269, in apply_gradients  **
        self.apply(grads, trainable_variables)
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 334, in apply
        variable.assign(variable.constraint(variable))
    File "c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\my_dnn.py", line 65, in __call__
        return self.apply_l1_projection(w)
    File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\tensorflow\core\function\polymorphism\function_type.py", line 583, in canonicalize_to_monomorphic
        _make_validated_mono_param(name, arg, poly_parameter.kind,
   

In [11]:
print(mses)

[0.95123291015625]


In [5]:
model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=10,
        num_layers=10,
        num_neurons=20,
        beta=80,
        gamma=80,
        delta=1,
    )

optimizer = keras.optimizers.SGD(learning_rate=np.exp(-14))

model.compile(
    optimizer=optimizer, loss="mean_squared_error"
)

model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10

1/1 [==============================] - 183s 183s/step - loss: 24.3104 - val_loss: 18.9069
Epoch 2/10
1/1 [==============================] - 0s 149ms/step - loss: 21.3919 - val_loss: 16.4487
Epoch 3/10
1/1 [==============================] - 0s 107ms/step - loss: 18.8508 - val_loss: 14.3193
Epoch 4/10
1/1 [==============================] - 0s 96ms/step - loss: 16.6383 - val_loss: 12.4755
Epoch 5/10
1/1 [==============================] - 0s 92ms/step - loss: 14.7118 - val_loss: 10.8795
Epoch 6/10
1/1 [==============================] - 0s 134ms/step - loss: 13.0343 - val_loss: 9.4987
Epoch 7/10
1/1 [==============================] - 0s 131ms/step - loss: 11.5736 - val_loss: 8.3048
Epoch 8/10
1/1 [==============================] - 0s 107ms/step - loss: 10.3016 - val_loss: 7.2730
Epoch 9/10
1/1 [==============================] - 0s 93ms/step - loss: 9.1940 - val_loss: 6.3818
Epoch 10/10
1/1 [==============================] - 0s 116ms/step - loss: 8.2294 - val_loss: 5.6125


In [11]:
x_val, y_val = get_data(m1, x_dim=m1.expected_dim, num_samples=10**5)
validation_data = preprocess(x_val, y_val, training=False)

model.evaluate(validation_data)

1563/1563 [==============================] - 26s 17ms/step - loss: 0.0071 - mean_squared_error: 0.0071


[0.007052791304886341, 0.007052791304886341]

In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

In [53]:

def apply_l1_projection(self, w):
    if tf.norm(w, ord=1) <= self:
        return w
    # Apply L1-projection on weight vector w
    abs_w = tf.abs(w)

    # Compute cumulative sum of the sorted absolute weights
    u = tf.sort(abs_w, direction="DESCENDING", axis=0)
    svp = tf.cumsum(u, axis=0) - self
    print(u.numpy())
    print(svp)

    # Find the position where the condition is violated for the first time
    ratio = tf.math.divide(svp, tf.constant(tf.range(1, tf.size(u) + 1, dtype=tf.float32), shape=tf.shape(u)))
    print(ratio.numpy())

    # Compute the threshold value
    theta = tf.reduce_max(tf.maximum(ratio, 0.0), axis=0)
    print(theta.numpy())

    return tf.math.sign(w) * tf.maximum(abs_w - theta, 0)

In [ ]:
w = tf.Variable(tf.ones(shape=(11, 1)), dtype=tf.float32)
apply_l1_projection(7.0, w)

In [13]:
def test_norm_l1(model):
    # Test if L1 projection of last layer worked
    weights = tf.reshape(model.trainable_variables[-1], [-1])
    norm = tf.norm(weights, ord=1)
    print(f"norm: {norm}, gamma: 80")


def test_norm_l2(model):
    # Test if L2 projection of inner weights worked
    current_weights = tf.concat(
        [tf.reshape(v, [-1]) for v in model.trainable_weights[:-1]], axis=0
    )
    sub_nets_init_weights = tf.concat(
        [tf.reshape(v, [-1]) for v in model.init_vars], axis=0
    )
    diff = sub_nets_init_weights - current_weights
    norm = tf.norm(diff)
    print(f"norm: {norm}, delta: 1")

In [14]:
test_norm_l1(model)
test_norm_l2(model)

norm: 0.06658446043729782, gamma: 80
norm: 8.979407721199095e-05, delta: 1


In [18]:
import numpy as np
def calculate_iqr(data):

    q75, q25 = np.percentile(data, [75, 25])
    return q75 - q25


def iqr_median(my_func, x_dim=1, num_samples=10**5, num_repetitions=100):
    """Function to calculate the IQR for a given function"""

    iqr_values = []
    rng = np.random.default_rng(42)

    for _ in range(num_repetitions):
        x_samples = rng.random((num_samples, x_dim))
        my_func_values = my_func(x_samples)
        iqr_values.append(calculate_iqr(my_func_values))
        print("-------------------------")
        print(x_samples)

    return np.median(iqr_values)

In [32]:
iqr_median(regression_func, x_dim=7, num_samples=10, num_repetitions=10)

-------------------------
[[0.77395605 0.43887844 0.85859792 0.69736803 0.09417735 0.97562235
  0.7611397 ]
 [0.78606431 0.12811363 0.45038594 0.37079802 0.92676499 0.64386512
  0.82276161]
 [0.4434142  0.22723872 0.55458479 0.06381726 0.82763117 0.6316644
  0.75808774]
 [0.35452597 0.97069802 0.89312112 0.7783835  0.19463871 0.466721
  0.04380377]
 [0.15428949 0.68304895 0.74476216 0.96750973 0.32582536 0.37045971
  0.46955581]
 [0.18947136 0.12992151 0.47570493 0.22690935 0.66981399 0.43715192
  0.8326782 ]
 [0.7002651  0.31236664 0.8322598  0.80476436 0.38747838 0.2883281
  0.6824955 ]
 [0.13975248 0.1999082  0.00736227 0.78692438 0.66485086 0.70516538
  0.78072903]
 [0.45891578 0.5687412  0.139797   0.11453007 0.66840296 0.47109621
  0.56523611]
 [0.76499886 0.63471832 0.5535794  0.55920716 0.3039501  0.03081783
  0.43671739]]
-------------------------
[[0.21458467 0.40852864 0.85340307 0.23393949 0.05830274 0.28138389
  0.29359376]
 [0.66191651 0.55703215 0.78389821 0.66431354 0.4

1.4402881839413224